In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam



In [ ]:
# Define function to load images and labels
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if img_path.endswith(".jpg") or img_path.endswith(".png"):  # assuming images are in jpg or png format
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # resize images for consistency
            images.append(img)
            labels.append(label)
    return images, labels

In [ ]:
# Load infected images
infected_folder = '/content/drive/MyDrive/PROJECT/PIT'
infected_images, infected_labels = load_images(infected_folder, 1)

# Load healthy images
healthy_folder = '/content/drive/MyDrive/PROJECT/PHT'
healthy_images, healthy_labels = load_images(healthy_folder, 0)


In [ ]:
# Concatenate infected and healthy images
X = np.concatenate((infected_images, healthy_images), axis=0)
y = np.concatenate((infected_labels, healthy_labels))

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


In [ ]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Siamese CNN model using VGG16 as base
input_left = Input(shape=(224, 224, 3))
input_right = Input(shape=(224, 224, 3))

output_left = base_model(input_left)
output_right = base_model(input_right)

# Concatenate outputs and add dense layers
merged_output = tf.keras.layers.Concatenate()([output_left, output_right])
flatten_layer = Flatten()(merged_output)
dropout_layer = Dropout(0.5)(flatten_layer)
dense_layer1 = Dense(128, activation='relu')(dropout_layer)
dense_layer2 = Dense(64, activation='relu')(dense_layer1)
output = Dense(1, activation='sigmoid')(dense_layer2)

siamese_model = Model(inputs=[input_left, input_right], outputs=output)
siamese_model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
# Data augmentation and training (limited to 10 epochs)
train_generator = datagen.flow([X_train, X_train], y_train, batch_size=32)
history = siamese_model.fit(train_generator, epochs=10, validation_data=([X_test, X_test], y_test))


Epoch 1/10
5/5 [==============================] - 233s 46s/step - loss: 1.1340 - accuracy: 0.5693 - val_loss: 0.1477 - val_accuracy: 0.9429
Epoch 2/10
5/5 [==============================] - 254s 53s/step - loss: 0.1329 - accuracy: 0.9562 - val_loss: 0.1271 - val_accuracy: 0.9714
Epoch 3/10
5/5 [==============================] - 260s 62s/step - loss: 0.0277 - accuracy: 0.9927 - val_loss: 0.1944 - val_accuracy: 0.9714
Epoch 4/10
5/5 [==============================] - 249s 52s/step - loss: 0.0650 - accuracy: 0.9781 - val_loss: 0.0265 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 208s 42s/step - loss: 0.0218 - accuracy: 0.9927 - val_loss: 0.0182 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 207s 41s/step - loss: 2.9621e-04 - accuracy: 1.0000 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 247s 52s/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0046 - val_accuracy: 1.0000
Epoch 8/10
5/5 [

In [ ]:
# Reshape labels to match the shape expected by the model
labels_list = labels_list.reshape(-1, 1, 1, 1)

# Train the model with data augmentation
siamese_model.fit(train_datagen.flow([pairs[:, 0], pairs[:, 1]], labels_list, batch_size=batch_size),
                  steps_per_epoch=len(pairs) / batch_size, epochs=10)

Epoch 1/10
5/5 [==============================] - 21s 3s/step - loss: 0.1023 - accuracy: 0.9722 
Epoch 2/10
5/5 [==============================] - 14s 3s/step - loss: 0.0096 - accuracy: 1.0000 
Epoch 3/10
5/5 [==============================] - 15s 4s/step - loss: 0.0048 - accuracy: 1.0000 
Epoch 4/10
5/5 [==============================] - 14s 3s/step - loss: 0.0032 - accuracy: 1.0000 
Epoch 5/10
5/5 [==============================] - 15s 3s/step - loss: 0.0026 - accuracy: 1.0000 
Epoch 6/10
5/5 [==============================] - 15s 3s/step - loss: 0.0021 - accuracy: 1.0000 
Epoch 7/10
5/5 [==============================] - 14s 3s/step - loss: 0.0018 - accuracy: 1.0000 
Epoch 8/10
5/5 [==============================] - 15s 3s/step - loss: 0.0016 - accuracy: 1.0000 
Epoch 9/10
5/5 [==============================] - 24s 6s/step - loss: 0.0015 - accuracy: 1.0000 
Epoch 10/10
5/5 [==============================] - 18s 3s/step - loss: 0.0015 - accuracy: 1.0000 


In [21]:
# Evaluate the model
y_pred = siamese_model.predict([X_test, X_test])
y_pred_binary = np.round(y_pred).flatten()

accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

2/2 [==============================] - 60s 3s/step
Accuracy: 0.9714285714285714
Precision: 1.0
Recall: 0.9444444444444444
F1 Score: 0.9714285714285714
